<a href="https://colab.research.google.com/github/EMAT31530/ai-group-project-music-classification/blob/master/Spotify_CNN_Playlist_Avg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
################## Playlist Dataset Google Drive Link (also in Google Drive of ai.music.generation@gmail.com ) ########################
https://drive.google.com/drive/folders/18it5Kuzlo5OPvcxvOxrve-7-bl9t4XeE?usp=sharing

In [43]:
import tensorflow as tf
import numpy as np
import pandas as pd
import scipy
from scipy import misc
import glob
from PIL import Image
import os
import matplotlib.pyplot as plt
import librosa
from keras import layers
from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, 
                          Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout)
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.initializers import glorot_uniform
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

import shutil
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img,img_to_array
import random
from google.colab import files

In [44]:
########### Load Trained Model - Doesn't seem to load properly ############
model = load_model("/content/drive/MyDrive/json_model/model2")
model.compile(optimizer = Adam(learning_rate=0.0005),loss='categorical_crossentropy',metrics=['accuracy'])

In [45]:
path = '/content/drive/MyDrive/playlist/images'
list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]
list_subfolder_name_with_paths = [f.name for f in os.scandir(path) if f.is_dir()]
list_subfolders_with_paths
list_subfolder_name_with_paths

['Corpse Pose',
 'All Along the Watchtower',
 'LEAVING HELL',
 'Love and or Hate Trigger',
 'Dsco',
 'Diary of a Rose',
 'Abusey Junction',
 'Circuits',
 'Like a Buoy',
 'George Street',
 'Strings of Light',
 'Suede',
 'All Caps',
 'Send Me',
 'Peanuts Grow Underground',
 'Blue in Green (feat. John Coltrane & Bill Evans)',
 'We Are Family',
 'Oblivion',
 'Slow (Loud)',
 'Track X',
 'Dark blue',
 'Bullet With a Name',
 'Wake Up',
 'Time',
 "Job's Lament",
 'Poison Tree',
 'A Message to You Rudy - 2002 Remaster',
 'Dirty Old Town',
 'Pressure Drop']

In [46]:
##########Dataset Load ##################
datasetpath = '/content/drive/MyDrive/playlist/playlist.csv'
df = pd.read_csv(datasetpath)


In [47]:
def predict(track_name,model):
  track_path = os.path.join(path, track_name)
  prediction = np.zeros((10,10))
  i = 0
  #image = image_data.resize((288,432))
  for file_name in os.listdir(track_path):
    file_path = os.path.join(path, track_name, file_name)
    image_data = load_img(file_path,color_mode='rgb',target_size=(300,300))
    image = img_to_array(image_data)

    image = np.reshape(image,(1,300,300,3))

    prediction[i,:] = model.predict(image/255)  #.reshape((10,)) 

    #prediction = prediction.reshape((10,)) 

    i += 1


  predict_avg = np.average(prediction, axis=0)
  predict_avg = np.around(predict_avg, decimals = 4)
  genre_class = np.argmax(predict_avg)




  
  return track_name, genre_class, predict_avg

In [48]:

track_name_list = []
genre_class_list = []
predict_avg_list = []

for subdir in os.listdir(path):
  track_name, genre_class, predict_avg = predict(subdir,model)
  print(track_name, genre_class, predict_avg)
  track_name_list.append(track_name)
  genre_class_list.append(genre_class)
  predict_avg_list.append(predict_avg)





Corpse Pose 8 [0.000e+00 0.000e+00 0.000e+00 4.000e-04 0.000e+00 3.000e-04 0.000e+00
 3.000e-04 9.403e-01 5.870e-02]
All Along the Watchtower 9 [0.000e+00 0.000e+00 0.000e+00 1.000e-04 0.000e+00 1.000e-04 0.000e+00
 0.000e+00 3.064e-01 6.933e-01]
LEAVING HELL 9 [0.     0.     0.     0.0234 0.     0.     0.     0.     0.028  0.9486]
Love and or Hate Trigger 9 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Dsco 9 [0.     0.     0.     0.0012 0.     0.     0.     0.     0.0023 0.9965]
Diary of a Rose 9 [0.     0.     0.     0.0015 0.     0.     0.     0.     0.05   0.9485]
Abusey Junction 9 [0.     0.     0.     0.     0.     0.     0.     0.     0.1769 0.8231]
Circuits 9 [0.000e+00 0.000e+00 0.000e+00 7.000e-04 0.000e+00 0.000e+00 0.000e+00
 1.000e-04 4.000e-04 9.989e-01]
Like a Buoy 9 [0.000e+00 0.000e+00 0.000e+00 2.630e-02 0.000e+00 1.000e-04 0.000e+00
 1.000e-04 4.625e-01 5.110e-01]
George Street 9 [0.     0.     0.     0.0071 0.     0.     0.     0.     0.1025 0.8904]
Strings of Light 9 [0.    0. 

In [49]:
########## Predict and Add Results to a Datafram ################
df_tracks = pd.DataFrame({'track_name':track_name_list,'genre_class':genre_class_list,'predict_avg':predict_avg_list})
print(df_tracks.shape)
df_tracks.head()

(29, 3)


,track_name,genre_class,predict_avg
0,Corpse Pose,8,"[0.0, 0.0, 0.0, 0.0004, 0.0, 0.0003, 0.0, 0.00..."
1,All Along the Watchtower,9,"[0.0, 0.0, 0.0, 0.0001, 0.0, 0.0001, 0.0, 0.0,..."
2,LEAVING HELL,9,"[0.0, 0.0, 0.0, 0.0234, 0.0, 0.0, 0.0, 0.0, 0...."
3,Love and or Hate Trigger,9,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,Dsco,9,"[0.0, 0.0, 0.0, 0.0012, 0.0, 0.0, 0.0, 0.0, 0...."


In [ ]:
df.to_csv('predictions.csv')
files.download('predictions.csv')